In [1]:
import numpy as np
import math
import pandas as pd

## Shell and Tube Heat Exchanger Design

#### Shell side and Tube side specification
1. component 1 will go to shell side and component 2 will go to tube side. 
2. Choose corrosive fluid as component 2
3. Specifications are as follows - 
  * Tin - Inlet Temperature (F)
  * Tout - Inlet Temperature (F)
  * ms - Mass Flowrate, (lb/hr)
  * cp - Specific Heat (Btu/lbF)
  * rho - Density of fluid (lb/ft3)
  * mu - Viscosity of fluid (cp)
  * K - thermal conductivity (Btuft/hr-ft^2-F)
  * fs - Fouling Allowance (hr-ft^2-F/Btu) 
  * ps - Inlet Pressure (psig)
  * dps - Allowable Pressure Drop (psi)

In [2]:
# Component 1  -  shell side specification

Tsin = 205          # (F)    Shell side inlet temp
Tsout = 264         # (F)    Shell side outlet temp

comp1_ms = 4000     # (lb/hr)
comp1_cp = 0.595    # (Btu/lbF)
comp1_rho = 40.6    # (lb/ft^3)
comp1_mu = 0.2605   # (cP)
comp1_k = 0.07      # (Btu/hr.ft.F)
comp1_fs = 0.002     # (hr-ft^2-F/Btu) Fouling Allowance 
comp1_ps = 180      # (psig), Inlet Pressure
comp1_dps = 5       # (psi), Allowable Pressure Drop


# Component 2  -  Tube side specification
Ttin = 620          # (F)    Shell side inlet temp
Ttout = 250         # (F)    Shell side outlet temp

comp2_ms = 590         # (lb/hr)
comp2_cp = 0.6615      # (Btu/lbF)
comp2_rho = 46.3       # (lb/ft^3)
comp2_mu = 1.3         # (cP)
comp2_k = 0.055        # (Btu/hr.ft.F)
comp2_ft = 0.005       # (hr-ft^2-F/Btu) Fouling Allowance 
comp2_pt = 65          # (psig), Inlet Pressure
comp2_dpt = 5          # (psi), Allowable Pressure Drop


#### Estimate Layouts
| Variables |
|------|
|Material use for Shell  |  
|Material use for Tubel  |
|Tube Outer Diameter (in)   |
|Tube Length (ft)  |
|Pitch (in)  |
|Tube Inner Diameter (in)  |
|Shell pass |
|Tube pass |

In [3]:
Material_Shell = 'carbon steel'
Material_Tube =  'carbon steel'
TDo = 0.75                         # (in) Tube Outer Diameter
TDin  =  0.5837                    # (in) Tube Inner Diameter
TL = 8                             # (ft) Tube length
TP= 1                              # (in) Pitch
Shell_Pass = 1
Tube_Pass = 2                 

#### Estimate Overall Heat Transfer Coefficient

Estimate overall heat transfer coefficient,
Uo $\frac{Btu}{hrft^{2}F^{\circ}}$

In [72]:
Uo = 6.75

#### Shell side Specification
1. Find the constants k1 and n1 from Pitch table 
2. Bs - Bundle shell clearance
3. Baffle_sp - Baffle spacing

In [73]:
k1 = 0.156
n1 =  2.291  
Bs =  2         #(in) Bundle-Shell Clearence
Baffle_sp = 2.8    # (in) Baffle spacing 


#### Overall Heat Transfer Coefficient Calculation
1. Jh - heat transfer factor 
2. Kw - Thermal conductivity of wall material

In [74]:
Jh_t =  0.07   # Heat Transfer Factor for tube side
Jh_s =  0.015  # Heat Transfer Factor for shell side
Kw = 30      # (Btu/hr-ft-F)

#### Shell Side and Tube Side Pressure Drop
* The value of the friction factor, Jf, will depend on the design of plate used. 
* For preliminary calculations the following relationship can be used for turbulent flow:
  $ J_{f} = 0.6(NRe)^{-0.3}$

In [75]:
Jf_T = 0.095
Jf_S = 0.05

### Code for the Heat exchanger design 

In [166]:
class Heatex:
    
    def __init__(self, c1_ms,c1_cp,c1_rho,c1_mu, c1_k, shell_inlet, shell_outlet, c2_ms,c2_cp,c2_rho,c2_mu, c2_k, tube_inlet, tube_outlet):
        self.shell_inlet = shell_inlet
        self.shell_outlet = shell_outlet
        self.tube_inlet = tube_inlet
        self. tube_outlet =  tube_outlet
        self.c1_ms = c1_ms
        self.c1_cp = c1_cp
        self.c1_rho = c1_rho
        self.c1_mu = c1_mu
        self.c1_k = c1_k
        self.c2_ms = c2_ms
        self.c2_cp = c2_cp
        self.c2_rho = c2_rho
        self.c2_mu = c2_mu
        self.c2_k = c2_k

    def Heat_duty(self):
        Q1 = abs((self.c1_ms)*(self.c1_cp)*(self.shell_inlet - self.shell_outlet))
        Q2 = abs((self.c2_ms)*(self.c2_cp)*(self.tube_inlet - self.tube_outlet))
        return Q1, Q2
    
 
   
    def LMTD(self):
        return abs(((self.shell_inlet - self.tube_outlet) - (self.shell_outlet - self.tube_inlet)) / (math.log((self.shell_inlet - self.tube_outlet)/(self.shell_outlet - self.tube_inlet))))
    
    
    
    def Correction_factor(self):
        
        """
       The correction factor Ft is a function of the shell and tube fluid temperatures,
       and the number of tube and shell passes.
       
        """
        
        R =  abs((self.tube_outlet - self.tube_inlet)/(self.shell_inlet - self.shell_outlet))
        S =  abs((self.shell_inlet - self.shell_outlet)/(self.tube_inlet - self.shell_inlet))
        
        A = (math.sqrt(R*R +1))*(np.log((1-S)/(1-R*S)))
        a = (2-S)*(R + 1 - math.sqrt(R*R +1))
        b = (2-S)*(R + 1 + math.sqrt(R*R +1))
        B = (R-1)*(np.log(a/b))
        
        
        return abs(A/B)
    
    
    def MTD(self):
        return Ft*Tlmtd
    
    def No_Tubes(self, U, D, L, Din, SP, TP, TFw, TFrho ):
        self.U = U
        self.D = D
        self.L = L
        self.Din = Din
        self.SP = SP
        self.TP = TP
        self.TFw = TFw
        self.TFrho = TFrho
        
        Ao = QS/(self.U*Tclmtd)
        Tube_Area  = math.pi*(D/12)*L
        No_TubePass = int((Ao/Tube_Area)/TP)
        TA = (math.pi/4)*((self.Din/12)**2)
        Area_PerPass = TA*No_TubePass
        Vol_F = (self.TFw/3600)*(1/self.TFrho)
        
        Tube_VolF = Vol_F / Area_PerPass
        
        return Tube_Area, int(Ao/Tube_Area), TA, No_TubePass, Area_PerPass, Vol_F, Tube_VolF
    
    def Bundle(self,Do, K1, N1, BSC):
        self.Do =Do
        self.K1 = K1
        self.N1 = N1
        self.BSC = BSC
        
        Bundle_dia = (Do)*((NOT/K1)**(1/N1))
        shell_dia = Bundle_dia + BSC
        return Bundle_dia, shell_dia
    
    
    
    def TubeHeatTransfer_coeff(self, JHt):
        self.JHt = JHt
        NRe_T = (self.c2_rho*TSV*3600*(self.Din/12))/(self.c2_mu*2.42)
        Pr_T = (self.c2_cp*self.c2_mu*2.42)/self.c2_k
        Nu_T = JHt*NRe_T*(Pr_T**(0.33))
        hi = Nu_T*(self.c2_k/(self.Din/12))
        return NRe_T, Pr_T, Nu_T, hi
    
    def Shell_vel(self, BS, Tp, JHs):
        
        self.BS = BS
        self.Tp = Tp
        self.JHs = JHs
        As = (((Tp - self.Do)/Tp)*SD*BS)/144
        Vol_flow = (self.c1_ms/3600)*(1/self.c1_rho)
        shell_vel = Vol_flow/As
        
        EQ_dia = (1.27/self.Do)*((Tp**2)-0.785*(self.Do**2))
        NRe_S  = (self.c1_rho*shell_vel*3600*(EQ_dia /12))/(self.c1_mu*2.42)
        Pr_S   = (self.c1_cp*self.c1_mu*2.42)/self.c1_k
        Nu_S   = JHs*NRe_S*(Pr_S**(0.33))
        hs     =   Nu_S*(self.c1_k/(EQ_dia/12))
        return As, Vol_flow ,shell_vel,EQ_dia, NRe_S, Pr_S, Nu_S,hs

    
    def OverallHTC(self, KW, c1_fs, c1_ps, c1_dps , c2_ft, c2_pt, c2_dpt):
        self.KW = KW
        self.c1_fs  = c1_fs
        self.c1_ps  = c1_ps
        self.c1_dps = c1_dps
        self.c2_ft  = c2_ft 
        self.c2_pt  = c2_pt
        self.c2_dpt = c2_dpt
        
        Uo = 1/((((1/HiT) + c2_ft)*(self.D/self.Din))+ (((self.D/12)*(math.log(self.D/self.Din)))/(2*KW))+(1/HiS)+c1_fs)
        
        return Uo
    
    
    def PressureDrop(self, Jf_t, Jf_s):
        
        self.Jf_t =Jf_t
        self.Jf_s =Jf_s
        
        TS_PD =  self.TP*(8*Jf_t*((self.L/3.28)/(self.Din/(12*3.28)))+2.5)*(((self.c2_rho*16.01)*(TSV/3.28)**2)/2)
        
        SS_PD =  8*Jf_s*((SD/12*3.28)/(EQD/(12*3.28)))*((self.L/3.28)/(self.BS/12*3.28))*((self.c1_rho*16.01)*(SSV/3.28)**2/2)
        
        return TS_PD,SS_PD
    
    def Conclusion(self):
        
        x = ((uo- self.U)/self.U )
        if 0 < x < 0.05 :  
            print("Specifications are correct")
            
        else :
            print("You need to change the inputs")
       
   
   


In [167]:
P = Heatex(comp1_ms,comp1_cp, comp1_rho, comp1_mu, comp1_k, Tsin, Tsout,comp2_ms,comp2_cp, comp2_rho, comp2_mu,comp2_k, Ttin, Ttout)
QS,QT = P.Heat_duty()
Tlmtd= P.LMTD()
Ft = P.Correction_factor()
Tclmtd = P.MTD()
AOT, NOT, TCA , NTP, EOP, VolFlow, TSV = P.No_Tubes(Uo, TDo, TL, TDin, Shell_Pass, Tube_Pass,comp2_ms, comp2_rho )
BD, SD = P.Bundle(TDo, k1, n1, Bs)
NReT, PrT, NuT, HiT =P.TubeHeatTransfer_coeff(Jh_t)
CSA, SVF, SSV, EQD, NReS, PrS, NuS, HiS = P.Shell_vel(Baffle_sp, TP, Jh_s)
uo = P.OverallHTC(Kw, comp1_fs, comp1_ps, comp1_dps, comp2_ft, comp2_pt, comp2_dpt)
TSPD, SSPD = P.PressureDrop(Jf_T, Jf_S)

In [168]:
A0 = {
    'Variable':['Heat duty - Tube side','Heat duty -  shell side','LMTD','Corrected LMTD'],
    'Result':[QS, QT, Tlmtd, Tclmtd],
    'Unit' : ['Btu/hr', 'Btu/hr', '$F^{\circ}$', ' $F^{\circ}$']}
df0 = pd.DataFrame(A0,columns=['Variable','Result', 'Unit'])




A1 = {
    'Variable':['Area of one Tube','Number of Tubes','Area per pass','Number of Tubes per pass','Tube cross-section area','Volumetric Flowrate','Tube side Velocity'],
    'Result':[AOT,NOT,EOP, NTP,TCA ,VolFlow, TSV],
    'Unit' : ['$ft^2$',' ','$ft^2$','','$ft^2$',' $ft^{3}s^{-1}$','$ft s^{-1}$']}
df1 = pd.DataFrame(A1,columns=['Variable','Result', 'Unit'])




A2 = {
    'Variable':['Bundle Diameter','Shell Diameter','Shell Side Cross Flow area','Volumetric Flowrate','Shell Side Velocity'],
    'Result':[BD, SD, CSA, SVF ,  SSV],
    'Unit' : ['$in$', '$in$', '$ft^2$', '$ft^{3}s^{-1}$', '$fts^{-1}$']}
df2 = pd.DataFrame(A2,columns=['Variable','Result', 'Unit'])


A3 = {
    'Variable':['Tube Side Pressure Drop','Tube Side Pressure Drop'],
    'Result':[ TSPD, SSPD],
    'Unit' : ['$Pa$', '$Pa$']}
df3 = pd.DataFrame(A3,columns=['Variable','Result', 'Unit'])


A4 = {
    'Variable':['Tube Side Heat Transfer Coefficient','Shell Side Heat Transfer Coefficient', 'Overall Heat Transfer Coefficient'],
    'Result':[ HiT, HiS, uo],
    'Unit' : ['$Btu/hrft^2F$', '$Btu/hrft^{2}F$', '$Btu/hrft^2F$']}
df4 = pd.DataFrame( A4, columns=['Variable','Result', 'Unit'])

In [169]:
print('   ')
print('\033[1m' "Heat Duty and LMTD")
df0

   
Heat Duty and LMTD


,Variable,Result,Unit
0,Heat duty - Tube side,140420.000000,Btu/hr
1,Heat duty - shell side,144405.450000,Btu/hr
2,LMTD,150.367343,$F^{\circ}$
3,Corrected LMTD,139.067022,$F^{\circ}$


In [170]:
print('   ')
print('\033[1m' "Tube Side - No of Tubes")
df1

   
Tube Side - No of Tubes


,Variable,Result,Unit
0,Area of one Tube,1.570796,$ft^2$
1,Number of Tubes,95.000000,
2,Area per pass,0.087338,$ft^2$
3,Number of Tubes per pass,47.000000,
4,Tube cross-section area,0.001858,$ft^2$
5,Volumetric Flowrate,0.003540,$ft^{3}s^{-1}$
6,Tube side Velocity,0.040529,$ft s^{-1}$


In [171]:
print('   ')
print('\033[1m' "Shell Side - Shell diameter and Shell side velocity")
df2

   
Shell Side - Shell diameter and Shell side velocity


,Variable,Result,Unit
0,Bundle Diameter,12.317217,$in$
1,Shell Diameter,14.317217,$in$
2,Shell Side Cross Flow area,0.069598,$ft^2$
3,Volumetric Flowrate,0.027367,$ft^{3}s^{-1}$
4,Shell Side Velocity,0.393222,$fts^{-1}$


In [172]:
print('   ')
print('\033[1m' "Pressure Drop")
df3

   
Pressure Drop


,Variable,Result,Unit
0,Tube Side Pressure Drop,14.429391,$Pa$
1,Tube Side Pressure Drop,969.904669,$Pa$


In [173]:
print('   ')
print('\033[1m' "Overall Heat Transfer Coefficient")
df4

   
Overall Heat Transfer Coefficient


,Variable,Result,Unit
0,Tube Side Heat Transfer Coefficient,27.419841,$Btu/hrft^2F$
1,Shell Side Heat Transfer Coefficient,166.577333,$Btu/hrft^{2}F$
2,Overall Heat Transfer Coefficient,16.247125,$Btu/hrft^2F$


#### Conclusion check

In [175]:
P.Conclusion()

You need to change the inputs
